In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
import subprocess
import os
# python version: 3.11.9

In [ ]:
# presets, functions

# preset values for the analysis:
#____________________________________________________________________________________________________________________
# physical constants
Z = 26
A = 56
mass_nucleon = 0.938273 
# mass_nucleus = 11.178
mass_nucleus = A * 0.931494
alpha_fine = 1 / 137

# three-momentum bin centers
qvcenters = [0.100, 0.148, 0.167, 0.205, 0.240, 0.300, 0.380, 0.475, 0.570, 0.649, 0.756, 0.991, 1.619, 1.921, 2.213, 2.500, 2.783, 3.500]
# three-momentum edges
qvbins = [0.063, 0.124, 0.158, 0.186, 0.223, 0.270, 0.340, 0.428, 0.523, 0.609, 0.702, 0.878, 1.302, 1.770, 2.067, 2.357, 2.642, 2.923, 4.500]
# four-momentum squared bin names, in string format
qvbin_names = ['[0.063,0.124]', '[0.124,0.158]', '[0.158,0.186]', '[0.186,0.223]', '[0.223,0.270]', '[0.270,0.340]', '[0.340,0.428]', '[0.428,0.523]', '[0.523,0.609]',
                '[0.609,0.702]', '[0.702,0.878]', '[0.878,1.302]', '[1.302,1.770]', '[1.770,2.067]', '[2.067,2.357]', '[2.357,2.642]', '[2.642,2.923]', '[2.923,4.500]']

# four-momentum squared bin centers
Q2centers = [0.010, 0.020, 0.026, 0.040, 0.056, 0.093, 0.120, 0.160, 0.265, 0.380, 0.500, 0.800, 1.250, 1.750, 2.250, 2.750, 3.250, 3.750]
# four-momentum squared bin edges
Q2bins = [0.004, 0.015, 0.025, 0.035, 0.045, 0.070, 0.100, 0.145, 0.206, 0.322, 0.438, 0.650, 1.050, 1.500, 2.000, 2.500, 3.000, 3.500, 4.000]
# four-momentum squared bin names, in string format
Q2bin_names = ['[0.004,0.015]', '[0.015,0.025]', '[0.025,0.035]', '[0.035,0.045]', '[0.045,0.070]', '[0.070,0.100]', '[0.100,0.145]', '[0.145,0.206]', '[0.206,0.322]',
                '[0.322,0.438]', '[0.438,0.650]', '[0.650,1.050]', '[1.050,1.500]', '[1.500,2.000]', '[2.000,2.500]', '[2.500,3.000]', '[3.000,3.500]', '[3.500,4.000]']

qv_Ex_increments = np.array([0.0012,0.001,0.0006,0.0006,0.0006,0.0010,0.0036,0.0036,0.0036,0.0036,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002])
qv_W2_increments = np.array([0.01,0.002,0.002,0.014,0.018,0.01,0.008,0.032,0.06,0.018,0.04,0.12,0.12,0.12,0.12,0.24,0.24,0.24])
q2_Ex_increments = np.array([0.006,0.0025,0.005,0.005,0.003,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005])
q2_W2_increments = np.array([0.01,0.01,0.01,0.008,0.01,0.008,0.008,0.03,0.015,0.05,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1])

def round_sig_3(x):
    return '%s' % float('%.3g' % x)

def append_row(df, row):
    return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)

def linear_model(x, a, b):
    return a * x + b

# read dataframe from csv file
def prepare_df(df):

    df["Veff"] = 0.0089
    
    df["Eeff"] = df["E0"] + df["Veff"]
    df["Ep"] = df["E0"] - df["nu"]
    df["Ep_eff"] = df["Ep"] + df["Veff"]

    # calculate normalized cross section:
    df['normalization'] = 1.0
    df['normError'] = 0.0
    df['system_err'] = 0.0
    df_norm = pd.read_csv('Data/Fe56_Norm.csv')
    for _, row in df_norm.iterrows():
        mask = pd.Series([True] * len(df))
        if pd.notna(row['E0']):
            mask &= df['E0'] == row['E0']
        if pd.notna(row['ThetaDeg']):
            mask &= df['ThetaDeg'] == row['ThetaDeg']
        if pd.notna(row['dataSet']):
            mask &= df['dataSet'] == row['dataSet']
        df.loc[mask, 'normalization'] = row['normalization']
        try:
            df.loc[mask, 'system_err'] = float(row['system_err'])
        except (ValueError, TypeError):
            df.loc[mask, 'system_err'] = np.nan
    df.loc[df['dataSet'] == 'Meziani:1984is', 'system_err'] = np.where(df.loc[df['dataSet'] == 'Meziani:1984is', 'Ep_eff'] > 0.2, 0.037, 0.037 + 0.016 * (0.2 - df.loc[df['dataSet'] == 'Meziani:1984is', 'Ep_eff']) / 0.15)
    df.loc[df['dataSet'] == 'Altemus:1973', 'system_err'] = np.where(df.loc[df['dataSet'] == 'Altemus:1973', 'Ep_eff'] > 0.2, 0.037, 0.037 + 0.016 * (0.2 - df.loc[df['dataSet'] == 'Altemus:1973', 'Ep_eff']) / 0.15)
    df['normCross'] = df['cross'] * df['normalization']
    df['normCrossError'] = (np.sqrt(df['error']**2 + ((df['system_err'] * df['cross'])**2))) * df['normalization']
    print(df.loc[df['normalization'] == 1, 'dataSet'].unique())
    
    # calculate the kinematic variables:
    df["ThetaRad"] = df["ThetaDeg"] * np.pi / 180
    df["sin2(T/2)"] = (np.sin(df["ThetaRad"] / 2))**2
    df["cos2(T/2)"] = (np.cos(df["ThetaRad"] / 2))**2
    df["tan2(T/2)"] = (np.tan(df["ThetaRad"] / 2))**2

    df["nuel"] = df["E0"] - df["E0"] / (1 + 2 * df["E0"] * df["sin2(T/2)"] / mass_nucleus)
    df["Ex"] = df["nu"] - (df["E0"] - df["E0"] / (1 + 2 * df["E0"] * df["sin2(T/2)"] / mass_nucleus))

    df["R"] = 1.1 * (df["A"])**(1/3) + 0.86 / ((df["A"])**(1/3))

    df["F2foc"] = (df["Eeff"] / df["E0"])**2
    df["Q2"] = 4 * df["E0"] * (df["Ep"]) * df["sin2(T/2)"]
    df["Q2eff"] = 4 * df["Eeff"] * df["Ep_eff"] * df["sin2(T/2)"]
    df["qv2"] = df["nu"]**2 + df["Q2eff"]
    df["qv"] = np.sqrt(df["qv2"])
    
    df["W2_O"] = mass_nucleon**2 + 2 * mass_nucleon * df["nu"] - df["Q2"]
    df["W2"] = mass_nucleon**2 + 2 * mass_nucleon * df["nu"] - df["Q2eff"]
    df["epsilon"] = 1 / (1 + 2 * (1 + (df["nu"]**2) / df["Q2eff"]) * df["tan2(T/2)"])

    df["gamma"] = alpha_fine * df["Ep_eff"] * (df["W2"] - mass_nucleon**2) / (( 4 * ((np.pi)**2) * df["Q2eff"] * mass_nucleon * df["E0"]) * (1 - df["epsilon"]))
    df["Sig_R"] = df["normCross"] / df["gamma"]
    df["D_sig_R"] = df["error"] / df["gamma"]
    df["Sig_mott"] = 4 * (alpha_fine**2) * (df["Ep"]**2) * df["cos2(T/2)"] / (df["Q2"]**2)
    df["Sig_mott_eff"] = df["Sig_mott"] * df["E0"] / df["Eeff"]

    # Calculate the Rosenbluth quantity:
    df["H"] = (df["qv2"]**2) / (4 * (alpha_fine**2) * (df["Ep_eff"]**2) * (df["cos2(T/2)"] + 2 * (df["qv2"] / df["Q2eff"]) * df["sin2(T/2)"]))
    df["Hcc"] = df["H"] / df["F2foc"]
    df["Hstar_Sig(nb)"] = df["H"] * df["normCross"]
    df["Hstar_error(nb)"] = df["H"] * df["normCrossError"]
    df["Hstar_Sig(GeV)"] = df["Hstar_Sig(nb)"] / ((0.1973269**2) * 10000000)
    df["Hstar_error(GeV)"] = df["Hstar_error(nb)"] / ((0.1973269**2) * 10000000)
    df["Hcc_Sig(nb)"] = df["Hcc"] * df["normCross"]
    df["Hcc_error(nb)"] = df["Hcc"] * df["normCrossError"]
    df["Hcc_Sig(GeV)"] = df["Hcc_Sig(nb)"] / ((0.1973269**2) * 10000000)
    df["Hcc_error(GeV)"] = df["Hcc_error(nb)"] / ((0.1973269**2) * 10000000)
    # we will fit "Hcc_Sig(GeV)" vs "epsilon" to the linear model to get the Rosenbluth slope and intercept

    # subdivide the data into bins
    df['qvbin'] = 0
    df['qvcenter'] = 0
    df['Q2bin'] = 0
    df['Q2center'] = 0
    df["qvbin"] = pd.cut(x=df["qv"], bins=qvbins, labels=qvbin_names, right=True)
    df["qvcenter"] = pd.cut(x=df["qv"], bins=qvbins, labels=qvcenters, right=True)
    df['qvcenter'] = pd.to_numeric(df['qvcenter'])
    df["Q2bin"] = pd.cut(x=df["Q2eff"], bins=Q2bins, labels=Q2bin_names, right=True)
    df["Q2center"] = pd.cut(x=df["Q2eff"], bins=Q2bins, labels=Q2centers, right=True)
    df['Q2center'] = pd.to_numeric(df['Q2center'])
    df = df.dropna()
    # now every row of data has a bin (Q2/qv) label

    # # this is for bc_qv_ex and bc_q2_ex
    # df['ExA'] = np.where(df['Ex'] >= 0.04, df['Ex'], 0.04)
    # df['nuA'] = np.where(df['Ex'] >= 0.04, df['nu'], df['nu'] + 0.04 - df['Ex'])
    # df["Q2effA"] = 4 * df["Eeff"] * (df["Eeff"] - df['nuA']) * df["sin2(T/2)"]
    # df["qv2A"] = df["nuA"]**2 + df["Q2effA"]
    # df["epsilonA"] = 1 / (1 + 2 * (1 + (df["nuA"]**2) / df["Q2effA"]) * df["tan2(T/2)"])

    qv_Ex_to_increment = dict(zip(qvcenters, qv_Ex_increments))
    qv_W2_to_increment = dict(zip(qvcenters, qv_W2_increments))
    Q2_Ex_to_increment = dict(zip(Q2centers, q2_Ex_increments))
    Q2_W2_to_increment = dict(zip(Q2centers, q2_W2_increments))

    df['qv_Ex_increment'] = df['qvcenter'].map(qv_Ex_to_increment)
    df['qv_Ex_start'] = 0.0
    df['qv_Ex_bin_index'] = np.floor((df['Ex'] - df['qv_Ex_start']) / df['qv_Ex_increment']).astype(int)
    df['Ex_qvcenter'] = df['qv_Ex_start'] + (df['qv_Ex_bin_index'] + 0.5) * df['qv_Ex_increment']

    df['qv_W2_increment'] = df['qvcenter'].map(qv_W2_to_increment)
    df['qv_W2_start'] = mass_nucleon**2 + 2 * mass_nucleon * 0.05 - (df['qvcenter']**2 - 0.05**2)
    df['qv_W2_bin_index'] = np.floor((df['W2'] - df['qv_W2_start']) / df['qv_W2_increment']).astype(int)
    df['W2_qvcenter'] = df['qv_W2_start'] + (df['qv_W2_bin_index'] + 0.5) * df['qv_W2_increment']

    df['Q2_Ex_increment'] = df['Q2center'].map(Q2_Ex_to_increment)
    df['Q2_Ex_start'] = 0.0
    df['Q2_Ex_bin_index'] = np.floor((df['Ex'] - df['Q2_Ex_start']) / df['Q2_Ex_increment']).astype(int)
    df['Ex_q2center'] = df['Q2_Ex_start'] + (df['Q2_Ex_bin_index'] + 0.5) * df['Q2_Ex_increment']

    df['Q2_W2_increment'] = df['Q2center'].map(Q2_W2_to_increment)
    df['Q2_W2_start'] = mass_nucleon**2 + 2 * mass_nucleon * 0.05 - df['Q2center']
    df['Q2_W2_bin_index'] = np.floor((df['W2'] - df['Q2_W2_start']) / df['Q2_W2_increment']).astype(int)
    df['W2_q2center'] = df['Q2_W2_start'] + (df['Q2_W2_bin_index'] + 0.5) * df['Q2_W2_increment']

    df['nu_Ex_qvcenter'] = - mass_nucleus + np.sqrt(mass_nucleus**2 + df['qvcenter']**2 + 2 * mass_nucleus * df['Ex_qvcenter'])
    df['nu_Ex_q2center'] = df['Ex_q2center'] + df['Q2center'] / (2 * mass_nucleus)
    df["epsilon_Ex_qvcenter"] = 1 / (1 + 2 * (1 + (df["nu_Ex_qvcenter"]**2) / (df["qvcenter"]**2 - df["nu_Ex_qvcenter"]**2)) * df["tan2(T/2)"])
    df["epsilon_Ex_q2center"] = 1 / (1 + 2 * (1 + (df["nu_Ex_q2center"]**2) / df["Q2center"]) * df["tan2(T/2)"])

    df['nu_W2_qvcenter'] = np.sqrt(df['qvcenter']**2 + df['W2_qvcenter']) - mass_nucleon
    df['nu_W2_q2center'] = (df['W2_q2center'] - mass_nucleon**2 + df['Q2center']) / (2 * mass_nucleon)
    df["epsilon_W2_qvcenter"] = 1 / (1 + 2 * (1 + (df["nu_W2_qvcenter"]**2) / (df["qvcenter"]**2 - df["nu_W2_qvcenter"]**2)) * df["tan2(T/2)"])
    df["epsilon_W2_q2center"] = 1 / (1 + 2 * (1 + (df["nu_W2_q2center"]**2) / df["Q2center"]) * df["tan2(T/2)"])

    df = df.dropna()

    return df

In [ ]:
# read csv file
df = pd.read_csv('Data/Fe56.csv')
df = prepare_df(df)
df = df[~((df['dataSet'] == "Chen:1990kq") & (df['E0'] == 2.7) & (df['W2'] < 1))]
df = df[~((df['dataSet'] == "Chen:1990kq") & (df['E0'] == 3.3) & (df['W2'] < 1))]
df = df[~((df['dataSet'] == "Chen:1990kq") & (df['E0'] == 3.6) & (df['W2'] < 1))]
df = df[~((df['dataSet'] == "Chen:1990kq") & (df['E0'] == 3.9) & (df['W2'] < 1))]
df = df[~((df['dataSet'] == "Chen:1990kq") & (df['E0'] == 4.3) & (df['W2'] < 0.8))]
df = df[~((df['dataSet'] == "Chen:1990kq") & (df['E0'] == 0.9) & (df['W2'] > 1))]
df = df[~((df['dataSet'] == "Chen:1990kq") & (df['E0'] == 1.1) & (df['W2'] > 1))]
df = df[~((df['dataSet'] == "Sealock:1989nx") & (df['E0'] == 1.299) & (df['W2'] < 0.75))]
df

In [ ]:
response_columns = ['i','RTTOT','RLTOT','RTQE','RLQE','RTNS','RLNS']

# calculate bc_qv_ex
df[['qvcenter','Ex_qvcenter']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_ex.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df['RL_qvc_ex'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvc_ex'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['qv','Ex']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_ex.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df['RL_qvd_ex'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvd_ex'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT qv_ex done.')

# calculate bc_qv_w2
df[['qvcenter','W2_qvcenter']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_w2.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df['RL_qvc_w2'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvc_w2'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['qv','W2']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_w2.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df['RL_qvd_w2'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvd_w2'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT qv_w2 done.')

# calculate bc_q2_ex
df[['Q2center','Ex_q2center']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_ex.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df['RL_q2c_ex'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2c_ex'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['Q2eff','Ex']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_ex.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df['RL_q2d_ex'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2d_ex'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT q2_ex done.')

# calculate bc_q2_w2
df[['Q2center','W2_q2center']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_w2.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df['RL_q2c_w2'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2c_w2'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['Q2eff','W2']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_w2.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df['RL_q2d_w2'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2d_w2'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT q2_w2 done.')


In [ ]:
# determine bin-centering corrections
df["bc_qv_ex"] = 1.0
df["bc_qv_w2"] = 1.0
df["bc_q2_ex"] = 1.0
df["bc_q2_w2"] = 1.0
for qvcenter in qvcenters:
    picked = df.loc[df["qvcenter"] == qvcenter]
    for index, row in picked.iterrows():
        if row['qvcenter'] != float('NaN'):
            try:
                df.loc[index, 'bc_qv_ex'] = (row['epsilon_Ex_qvcenter'] * row['RL_qvc_ex'] + 0.5 * ((qvcenter**2) / (qvcenter**2 - row['nu_Ex_qvcenter']**2)) * row['RT_qvc_ex']) / (row['epsilon'] * row['RL_qvd_ex'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_qvd_ex'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_qv_ex due to zero denominator.')
            try:
                df.loc[index, 'bc_qv_w2'] = (row['epsilon_W2_qvcenter'] * row['RL_qvc_w2'] + 0.5 * ((qvcenter**2) / (qvcenter**2 - row['nu_W2_qvcenter']**2)) * row['RT_qvc_w2']) / (row['epsilon'] * row['RL_qvd_w2'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_qvd_w2'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_qv_w2 due to zero denominator.')

for Q2center in Q2centers:
    picked = df.loc[df["Q2center"] == Q2center]
    for index, row in picked.iterrows():
        if row['Q2center'] != float('NaN'):
            try:
                df.loc[index, 'bc_q2_ex'] = (row['epsilon_Ex_q2center'] * row['RL_q2c_ex'] + 0.5 * ((Q2center + row["nu_Ex_q2center"]**2) / (Q2center)) * row['RT_q2c_ex']) / (row['epsilon'] * row['RL_q2d_ex'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_q2d_ex'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_q2_ex due to zero denominator.')
            try:
                df.loc[index, 'bc_q2_w2'] = (row['epsilon_W2_q2center'] * row['RL_q2c_w2'] + 0.5 * ((Q2center + row["nu_W2_q2center"]**2) / (Q2center)) * row['RT_q2c_w2']) / (row['epsilon'] * row['RL_q2d_w2'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_q2d_w2'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_q2_w2 due to zero denominator.')

In [ ]:
# save dataframe to csv
df.to_csv('Data/df_Fe56.csv',index=False)